In [1]:
import warnings
warnings.filterwarnings('ignore') # hide warnings
import os
import pandas as pd
import geopandas as gpd
import numpy as np
from dyntapy.supply_data import road_network_from_place, relabel_graph
from dyntapy.demand_data import add_centroids, od_graph_from_matrix
from dyntapy.assignments import StaticAssignment
from pyproj import Proj, transform
from dyntapy.toll import create_toll_object

ext = 2  # 0 or 2
buffer = 3
city = 'BRUSSEL'


network = road_network_from_place("Brussels", buffer_dist_close=buffer*1000)
network = relabel_graph(network)

composing
retrieved network graph for Brussels, with 3332 nodes and 6442 edges after processing


In [2]:
# string checks
HERE = os.path.dirname(os.path.realpath("__file__"))
data_path = HERE + os.path.sep +'STA_prep'

zoning_path = data_path + os.path.sep + "shapefile_data" + os.path.sep + city + "_" + str(buffer) + "_10" + os.path.sep + city + "_" + str(buffer) + "_10.shp"
od_path2 = data_path + os.path.sep + "od_matrix_data_ext_tr" + os.path.sep + city + "_ext_tr_" + str(buffer) + "_9_10.xlsx"
od_path0 = data_path + os.path.sep + "od_matrix_data" + os.path.sep + city + "_" + str(buffer) + "_9_.xlsx"

print('zoning path: ' + zoning_path)
print('od path (if ext = 2): ' + od_path2)
print('od path (if ext = 0): ' + od_path0)

# Read in the zoning file
HERE = os.path.dirname(os.path.realpath("__file__"))
data_path = HERE + os.path.sep +'STA_prep'
zoning_path = data_path + os.path.sep + "shapefile_data" + os.path.sep + city + "_" + str(buffer) + "_10" + os.path.sep + city + "_" + str(buffer) + "_10.shp"
zoning = gpd.read_file(zoning_path) # Everything is now stored in a geopandas dataframe. 

# Retrieve zone number, x_centroid (LON) and y_centroid (LAT) from each zone. 
zone_numbers = zoning["ZONENUMMER"]
x_lamb = zoning["X_LAMB"]
x_lamb = x_lamb.to_numpy()
y_lamb = zoning["Y_LAMB"]
y_lamb = y_lamb.to_numpy()

# Project to correct CRS. 
inProj = Proj(init='epsg:31370')
outProj = Proj(init='epsg:4326')
x_centroids, y_centroids = transform(inProj,outProj,x_lamb,y_lamb)

# Add the centroids to the network. Relabelling the graph is required (see demo for reason why)
connector_type = 'link'
network = add_centroids(network, x_centroids,y_centroids,k=1, method=connector_type)
network = relabel_graph(network)

# Read in the od matrix 
if ext == 2:
    od_path = data_path + os.path.sep + "od_matrix_data_ext_tr" + os.path.sep + city + "_ext_tr_" + str(buffer) + "_9_10.xlsx"
if ext == 0:
    od_path = data_path + os.path.sep + "od_matrix_data" + os.path.sep + city + "_" + str(buffer) + "_9_.xlsx"

od = pd.read_excel(od_path)
od_array = od.to_numpy() # The OD matrix is now stored in a numpy array

# Create OD graph and plot demand
od_graph = od_graph_from_matrix(od_array,x_centroids,y_centroids) 
# demand plot is useless since there are too many OD-pairs 

zoning path: C:\Users\anton\IP2\STA_prep\shapefile_data\BRUSSEL_3_10\BRUSSEL_3_10.shp
od path (if ext = 2): C:\Users\anton\IP2\STA_prep\od_matrix_data_ext_tr\BRUSSEL_ext_tr_3_9_10.xlsx
od path (if ext = 0): C:\Users\anton\IP2\STA_prep\od_matrix_data\BRUSSEL_3_9_.xlsx


In [4]:
assignment = StaticAssignment(network,od_graph)
result = assignment.run('dial_b')
print(result.link_costs[5:13])

init passed successfully
[0.00174699 0.0013169  0.00088443 0.00099331 0.00195037 0.00047927
 0.00169347 0.00049386]


In [5]:
toll_method = 'single'
toll_link_id = 9
toll_value = 10
toll_object = create_toll_object(network, toll_method, toll_link_id, toll_value)
assignment2 = StaticAssignment(network,od_graph, toll_object)
result2 = assignment2.run('dial_b')

init passed successfully


In [6]:
print(result2.link_costs[5:13])

[1.74698970e-03 1.31689987e-03 8.84434679e-04 9.93310569e-04
 1.00019504e+01 4.79273486e-04 1.69346719e-03 4.93862847e-04]


In [7]:
toll_method = 'cordon'
toll_link_id = [5, 9, 10]
toll_value = 10
toll_object2 = create_toll_object(network, toll_method, toll_link_id, toll_value)
assignment3 = StaticAssignment(network,od_graph, toll_object2)
result3 = assignment3.run('dial_b')
print(result3.link_costs[5:13])

init passed successfully
[1.00017470e+01 1.31689987e-03 8.84434679e-04 9.93310569e-04
 1.00019504e+01 1.00004793e+01 1.69346719e-03 4.93862847e-04]


In [8]:
toll_method = 'cordon'
toll_link_id = [5, 9, 10]
toll_value = [10, 20, 30]
toll_object3 = create_toll_object(network, toll_method, toll_link_id, toll_value)
assignment4 = StaticAssignment(network,od_graph, toll_object3)
result4 = assignment4.run('dial_b')
print(result4.link_costs[5:13])

init passed successfully
[1.00017470e+01 1.31689987e-03 8.84434679e-04 9.93310569e-04
 2.00019504e+01 3.00004793e+01 1.69346719e-03 4.93862847e-04]


In [9]:
toll_method = 'zone'
toll_link_id = [5, 9, 10,12]
toll_value = 3
toll_object4 = create_toll_object(network, toll_method, toll_link_id, toll_value)
assignment5 = StaticAssignment(network,od_graph, toll_object4)
result5 = assignment5.run('dial_b')
print(result5.link_costs[5:13])

init passed successfully
[1.04994006e+00 1.31689987e-03 8.84434679e-04 9.93310569e-04
 1.17217376e+00 2.88043350e-01 1.69346719e-03 2.96811574e-01]
